# Connect hybrid instance

In [ ]:
# this shouldn't run in a notebook, but in the unit tests...

import pytest
import lamindb_setup as ln_setup
from pathlib import Path

ln_setup.login("testuser1")
ln_setup.init(storage="s3://lamindb-ci/load_remote_instance")

assert ln_setup.settings.instance.name == "load_remote_instance"
assert ln_setup.settings.instance.storage.type_is_cloud
assert (
    ln_setup.settings.instance.storage.root_as_str
    == "s3://lamindb-ci/load_remote_instance"
)
assert (
    ln_setup.settings.instance._sqlite_file.as_posix()
    == f"s3://lamindb-ci/load_remote_instance/{ln_setup.settings.instance.id.hex}.lndb"  # noqa
)
with pytest.raises(ValueError) as error:
    ln_setup.settings.instance.local_storage
assert error.exconly() == "ValueError: Local storage is not enabled for this instance."
ln_setup.settings.instance._local_storage_on = True
with pytest.raises(ValueError) as error:
    ln_setup.settings.instance.local_storage
assert (
    error.exconly()
    == "ValueError: No local storage root found, set via"
    " `ln.setup.settings.instance.local_storage = local_root`"
)

# now set local storage location
ln_setup.settings.instance.local_storage = "./my_local_storage"

assert (
    ln_setup.settings.instance.local_storage.root.as_posix()
    == Path("./my_local_storage").resolve().as_posix()
)
assert ln_setup.settings.instance.local_storage is not None
with pytest.raises(ValueError) as error:
    ln_setup.settings.instance.local_storage = "./my_local_storage2"
assert error.exconly().startswith(
    "ValueError: You already configured a local storage root for this instance in this"
    " environment:"
)
ln_setup.delete("load_remote_instance", force=True)